In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import torch
import torch.nn as nn
from torchvision.models import resnet50
from torch.utils.data.dataloader import Dataset, DataLoader
from torch.optim.adam import Adam 
from torchvision import transforms
import torchvision
import cv2
from PIL import Image
import glob



In [ ]:
# 윈도우 cuda 설정
'''
device = "cuda" if torch.cuda.is_available() else "cpu"
device
'''

In [ ]:
# 맥 mps 설정
device = torch.device("mps:0" if torch.backends.mps.is_available() else "cpu")
print(f"현재 디바이스는 {device} 입니다.")

# 데이터 전처리

In [ ]:
trans = transforms.Compose([
    transforms.ToTensor()
])

In [ ]:
trainset = torchvision.datasets.ImageFolder(root = "data/x20/train_x20", transform=trans)

In [ ]:
imgs = [item[0] for item in trainset]

In [ ]:
imgs = torch.stack(imgs, dim=0).numpy()

In [ ]:
# 데이터 정규화를 위한 평균값
mean_r = imgs[:,0,:,:].mean()
mean_g = imgs[:,1,:,:].mean()
mean_b = imgs[:,2,:,:].mean()
print(mean_r, mean_g, mean_b)

In [ ]:
# 데이터 정규화를 위한 표준편차
std_r = imgs[:,0,:,:].std()
std_g = imgs[:,1,:,:].std()
std_b = imgs[:,2,:,:].std()
print(std_r, std_g, std_b)

In [ ]:
trans = transforms.Compose([transforms.Resize((1500,800)),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=(mean_r,mean_g,mean_b),
                                                 std = (std_r,std_g,std_b))
                            ])

# 데이터 불러오기

## 트레인 데이터 불러오기

In [ ]:
trainset = torchvision.datasets.ImageFolder(root = "data/x20/train_x20", transform=trans)

In [ ]:
len(trainset)

In [ ]:
classes = trainset.classes
classes

In [ ]:
trainloader = DataLoader(trainset, batch_size = 4, shuffle = True)

In [ ]:
images1, labels1 = next(iter(trainloader))
print(labels1)

# Validation 데이터 불러오기

In [ ]:
validation = torchvision.datasets.ImageFolder(root = "data/x20/val_x20", transform=trans)

In [ ]:
len(validation)

In [ ]:
validloader = DataLoader(validation, batch_size = 4, shuffle = False)

## 테스트 데이터 불러오기

In [ ]:
testset = torchvision.datasets.ImageFolder(root = "data/x20/test_x20", transform=trans)

In [ ]:
len(testset)

In [ ]:
classes = testset.classes
classes

In [ ]:
testloader = DataLoader(testset, batch_size = 4, shuffle = False)

In [ ]:
images2, labels2 = next(iter(testloader))
print(labels2)

# 모델 정의

In [ ]:
model = resnet50(weights = None)

In [ ]:
model.fc = nn.Sequential(
    nn.Linear(2048, 128),
    nn.ReLU(inplace = True),
    nn.Linear(128, 2))

In [ ]:
model.to(device)

In [ ]:
# 학습 루프 정의
lr = 1e-4
optim = Adam(model.parameters(), lr=lr)

for epoch in range(20):
    iterator = tqdm.tqdm(trainloader)
    for data, label in iterator:
        model.train()
        optim.zero_grad()
        preds = model(data.to(device))

        loss = nn.CrossEntropyLoss()(preds, label.to(device))
        loss.backward()
        optim.step()
        
        iterator.set_description(f"epoch:{epoch+1} loss:{loss.item()}")

torch.save(model.state_dict(), "ResNet_test.pth")

In [ ]:
model.load_state_dict(torch.load("ResNet_test.pth", map_location=device))

num_corr = 0

model.eval()
with torch.no_grad():
    for data, label in testloader:
        output = model(data.to(device))
        preds = output.data.max(1)[1]
        corr = preds.eq(label.to(device).data).sum().item()
        num_corr += corr
    print (f"Accuracy : {num_corr/len(testset)}")